In [ ]:
from keras.models import Model, Input
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D, Activation
from keras.utils import to_categorical
from keras.losses import categorical_crossentropy
from keras.callbacks import ModelCheckpoint, TensorBoard, ReduceLROnPlateau, EarlyStopping
from keras.optimizers import Adam
from sklearn.model_selection import train_test_split
import os
import pickle
import numpy as np

# Unpickle preprocessed data

In [ ]:
pkl_dir = os.path.join(os.getcwd(), '..', 'pkl')

def unpickle(path):
    with open(path, 'rb') as f:
        return pickle.load(f)

filepaths = [os.path.join(pkl_dir, filename) for filename in sorted(os.listdir(pkl_dir))]
data = []
for cls, filepath in enumerate(filepaths):
    feature_mats = unpickle(filepath)
    data.extend(feature_mats)

In [ ]:
img_tensors = []
for img_tensor in data:
    img_tensors.append(np.array(img_tensor))
X = np.array(img_tensors)

In [ ]:
X.shape

# Put together training and test tensors

In [ ]:
cls = -1
classes = []
for i in range(2100):
    if i % 100 == 0:
        cls += 1
    classes.append(cls)
X_train, X_test, y_train, y_test = train_test_split(X, classes, test_size=0.2, random_state=42)
y_train = to_categorical(y_train, num_classes=21)

# Model architecture

In [ ]:
input_shape = X_train[0,:,:,:].shape
model_input = Input(shape=input_shape)

In [ ]:
input_shape

In [ ]:
def conv_pool_cnn(model_input):
    
    x = Conv2D(96, (3, 3), activation='relu', padding = 'same', data_format='channels_first')(model_input)
    x = Conv2D(96, (3, 3), activation='relu', padding = 'same', data_format='channels_first')(x)
    x = MaxPooling2D(pool_size=(3, 3), strides = 2, data_format='channels_first')(x)
    x = Conv2D(192, (3, 3), activation='relu', padding = 'same', data_format='channels_first')(x)
    x = Conv2D(192, (3, 3), activation='relu', padding = 'same', data_format='channels_first')(x)
    x = MaxPooling2D(pool_size=(3, 3), strides = 2, data_format='channels_first')(x)
    x = Conv2D(192, (3, 3), activation='relu', padding = 'same', data_format='channels_first')(x)
    x = Conv2D(192, (1, 1), activation='relu', padding = 'same', data_format='channels_first')(x)
    x = Conv2D(21, (1, 1), data_format='channels_first')(x)
    x = GlobalAveragePooling2D(data_format='channels_first')(x)
    x = Activation(activation='softmax')(x)
    
    model = Model(model_input, x, name='conv_pool_cnn')
    
    return model


In [ ]:
conv_pool_cnn_model = conv_pool_cnn(model_input)

In [ ]:
conv_pool_cnn_model.summary()

# Model training

In [ ]:
def compile_and_train(model, num_epochs): 
    
    model.compile(loss=categorical_crossentropy, optimizer=Adam(lr=0.0001), metrics=['acc']) 
    filepath = 'weights/dl/' + model.name + '.{epoch:02d}-{loss:.2f}.hdf5'
    checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_weights_only=True,
                                                 save_best_only=True, mode='auto', period=1)
    tensor_board = TensorBoard(log_dir='logs/dl/', histogram_freq=0)
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5,
                                  patience=5, min_lr=0.0000001,
                                  verbose=1)
    early_stopping = EarlyStopping(monitor='val_acc', min_delta=0.01, patience=30, 
                                   verbose=1, mode='auto', restore_best_weights=True)
    history = model.fit(x=X_train, y=y_train, batch_size=32, shuffle=True,
                     epochs=num_epochs, verbose=1, callbacks=[checkpoint, tensor_board, reduce_lr, early_stopping], validation_split=0.2)
    return history



In [ ]:
_ = compile_and_train(conv_pool_cnn_model, num_epochs=200)

# Evaluation

In [ ]:
def evaluate_acc(model):
    pred = model.predict(X_test, batch_size = 32)
    pred = np.argmax(pred, axis=1)
    acc = np.sum(np.equal(pred, y_test)) / len(y_test)
    return acc


In [ ]:
pred = conv_pool_cnn_model.predict(X_test, batch_size = 32)
pred = np.argmax(pred, axis=1)
np.sum(np.not_equal(pred, y_test))

In [ ]:
evaluate_acc(conv_pool_cnn_model)

In [ ]:
def evaluate_top_n_acc(model, n):
    pred = conv_pool_cnn_model.predict(X_test, batch_size = 32)
    hit = 0
    for probs, truth in zip(pred, y_test):
        top_n = probs.argsort()[-n:][::-1]
        if truth in top_n:
            hit += 1
    acc = hit / len(y_test)
    return acc

In [ ]:
evaluate_top_n_acc(conv_pool_cnn_model, 5)